In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os


In [ ]:
from tensorflow.keras.models import load_model
garbageNet = load_model("garbage_classifier.h5")

In [ ]:
def detect_and_predict_garbage(frame, faceNet, garbageNet):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()

    faces = []
    locs = []
    preds = []

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > args["confidence"]:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))

    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = garbageNet.predict(faces, batch_size=32)

    return (locs, preds)


In [ ]:
args = {
    "garbage": "garbage_detector",
    "model": "garbage_classifier.h5",
    "confidence": 0.5
}


In [ ]:
print("[INFO] loading face detector model...")
prototxtPath = os.path.sep.join([args["garbage"], "deploy.prototxt"])
weightsPath = os.path.sep.join([args["garbage"],
    "res10_300x300_ssd_iter_140000.caffemodel"])
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [ ]:
print("[INFO] loading face mask detector model...")
garbageNet = load_model(args["model"])


In [ ]:
LABELS = ["Cardboard", "Glass", "Metal", "Paper", "Plastic", "Trash"]


In [ ]:
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

In [ ]:
while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    edged = cv2.Canny(blurred, 30, 150)
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        (x, y, w_box, h_box) = cv2.boundingRect(c)
        if w_box < 50 or h_box < 50:
            continue

        roi = frame[y:y+h_box, x:x+w_box]
        try:
            roi_resized = cv2.resize(roi, (224, 224))
        except:
            continue

        roi_resized = roi_resized.astype("float32") / 255.0
        roi_resized = np.expand_dims(roi_resized, axis=0)

        pred = garbageNet.predict(roi_resized)[0]
        class_index = np.argmax(pred)
        confidence = pred[class_index]

        if confidence > 0.7:
            label = LABELS[class_index]
            color = (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w_box, y + h_box), color, 2)
            cv2.putText(frame, f"{label}: {confidence*100:.1f}%", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("Garbage Detection", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break


In [ ]:
quit